<a href="https://colab.research.google.com/github/Bhavya418/DataMining_CP3/blob/main/Morning_Evening_Star.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
!pip install plotly
!pip install mplfinance

In [23]:
import pandas as pd
import mplfinance as mpf
import matplotlib.pyplot as plt
import requests
import plotly.graph_objects as go
from datetime import datetime

In [24]:
def fetch_daily_ohlc(symbol, apikey):
    url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&outputsize=compact&apikey={apikey}"
    r = requests.get(url)
    return r.json()

In [63]:
apikey = "SHEYWPL1ZYCDZ17D"
symbol = "RPOWER.BSE"
data = fetch_daily_ohlc(symbol, apikey)
print(data)

{'Meta Data': {'1. Information': 'Daily Prices (open, high, low, close) and Volumes', '2. Symbol': 'RPOWER.BSE', '3. Last Refreshed': '2025-04-23', '4. Output Size': 'Compact', '5. Time Zone': 'US/Eastern'}, 'Time Series (Daily)': {'2025-04-23': {'1. open': '43.8900', '2. high': '44.1800', '3. low': '42.3200', '4. close': '43.5500', '5. volume': '3519503'}, '2025-04-22': {'1. open': '44.2600', '2. high': '44.8900', '3. low': '43.2000', '4. close': '43.5400', '5. volume': '4283262'}, '2025-04-21': {'1. open': '42.2800', '2. high': '44.6500', '3. low': '42.0500', '4. close': '44.2100', '5. volume': '8797336'}, '2025-04-17': {'1. open': '41.9900', '2. high': '43.0000', '3. low': '41.6700', '4. close': '42.1400', '5. volume': '3834126'}, '2025-04-16': {'1. open': '41.8600', '2. high': '42.5500', '3. low': '41.4700', '4. close': '41.8100', '5. volume': '5141672'}, '2025-04-15': {'1. open': '40.0600', '2. high': '42.6000', '3. low': '40.0600', '4. close': '41.6300', '5. volume': '6385477'}, 

In [26]:
def is_red(candle):
    return float(candle["4. close"]) < float(candle["1. open"])

def is_green(candle):
    return float(candle["4. close"]) > float(candle["1. open"])

In [47]:
def is_doji_and_is_spinning_top(candle,body_thresold_pct =40):

    open_price = float(candle["1. open"])
    close_price = float(candle["4. close"])
    high_price = float(candle["2. high"])
    low_price = float(candle["3. low"])

    body = abs(close_price - open_price)
    total_range = abs(high_price - low_price)

    if total_range == 0:
      return False

    body_pct = (body/total_range) * 100

    return body_pct <= body_thresold_pct

In [28]:
def is_gap_down(prev_close,current_open):
    return current_open < prev_close

def is_gap_up(prev_close, current_open):
    return current_open > prev_close

In [29]:
def is_downtrend(candles):
    """True if lows are decreasing = downtrend"""
    low_prices = [float(c["3. low"]) for c in candles]
    for earlier, later in zip(low_prices, low_prices[1:]):
        if earlier <= later:
            return False
    return True

def is_uptrend(candles):
    """True if lows are increasing = uptrend"""
    low_prices = [float(c["3. low"]) for c in candles]

    for earlier, later in zip(low_prices, low_prices[1:]):
        if earlier >= later:
            return False
    return True

In [56]:
def is_morning_star(p1,p2,p3):

    p1_open = float(p1["1. open"])
    p1_close = float(p1["4. close"])
    p2_open = float(p2["1. open"])
    p2_close = float(p2["4. close"])
    p3_open = float(p3["1. open"])
    p3_close = float(p3["4. close"])

    if not is_red(p1):
      return False

    if not is_doji_and_is_spinning_top(p2):
      return False

    if not is_gap_down(p1_close, p2_open):
        return False

    if not is_green(p3):
      return False

    if not is_gap_up(p2_close, p3_open):
      return False

    if p3_close <= p1_open:
        return False

    return True


def is_evening_star(p1, p2, p3):
    p1_open = float(p1["1. open"])
    p1_close = float(p1["4. close"])
    p2_open = float(p2["1. open"])
    p2_close = float(p2["4. close"])
    p3_open = float(p3["1. open"])
    p3_close = float(p3["4. close"])

    if not is_green(p1):
        return False

    if not is_doji_and_is_spinning_top(p2):
        return False

    if not is_gap_up(p1_close, p2_open):
        return False

    if not is_red(p3):
        return False

    if not is_gap_down(p2_close, p3_open):
        return False

    if p3_close >= p1_open:
        return False

    return True


In [54]:
def detect_star(data, trend_days=3):
    time_series = data.get("Time Series (Daily)", {})
    sorted_dates = sorted(time_series.keys())
    results = []

    for i in range(trend_days + 2, len(sorted_dates)):
        p3_date = sorted_dates[i]
        p2_date = sorted_dates[i - 1]
        p1_date = sorted_dates[i - 2]
        trend_dates = sorted_dates[i - trend_days - 2 : i - 2]
        trend_candles = [time_series[d] for d in trend_dates]

        p1 = time_series[p1_date]
        p2 = time_series[p2_date]
        p3 = time_series[p3_date]

        pattern = None
        entry_price = float(p3["4. close"])
        stoploss = None

        if is_downtrend(trend_candles) and is_morning_star(p1, p2, p3):
            pattern = "morning_star"
            stoploss = min(
                float(p1["3. low"]),
                float(p2["3. low"]),
                float(p3["3. low"])
            )

        elif is_uptrend(trend_candles) and is_evening_star(p1, p2, p3):
            pattern = "evening_star"
            stoploss = max(
                float(p1["2. high"]),
                float(p2["2. high"]),
                float(p3["2. high"])
            )

        if pattern:
            results.append({
                "date": p3_date,
                "pattern": pattern,
                "entry_price": entry_price,
                "stoploss": stoploss,
                "open": float(p3["1. open"]),
                "high": float(p3["2. high"]),
                "low": float(p3["3. low"]),
                "close": float(p3["4. close"]),
                "volume": int(p3["5. volume"])
            })

    return pd.DataFrame(results)


In [58]:
def calculate_profit_loss_star(data, star_df, target_pct=2.0, hold_days=3):
    time_series = data.get("Time Series (Daily)", {})
    df = pd.DataFrame.from_dict(time_series, orient='index')
    df = df[['1. open', '2. high', '3. low', '4. close', '5. volume']].astype(float)
    df.columns = ['Open', 'High', 'Low', 'Close', 'Volume']
    df.index = pd.to_datetime(df.index)
    df = df.sort_index()

    trades = []
    total_profit_loss = 0.0

    for _, row in star_df.iterrows():
        entry_date = pd.to_datetime(row["date"])
        direction = row["pattern"]
        entry_price = row["entry_price"]

        if entry_date not in df.index:
            continue

        future_dates = df.index[df.index > entry_date]
        window_dates = future_dates[:hold_days]

        if len(window_dates) == 0:
            continue

        # Set target price
        target_price = (
            entry_price * (1 + target_pct / 100)
            if direction == "morning_star"
            else entry_price * (1 - target_pct / 100)
        )

        exit_price = None
        exit_date = None
        reason = ""

        # Step 1: Check for target hit
        for day in window_dates:
            high = df.loc[day]["High"]
            low = df.loc[day]["Low"]
            open_price = df.loc[day]["Open"]

            if direction == "morning_star" and high >= target_price:
                exit_price = open_price
                exit_date = day
                reason = "target_hit"
                break
            elif direction == "evening_star" and low <= target_price:
                exit_price = open_price
                exit_date = day
                reason = "target_hit"
                break

        # Step 2: If target not hit, exit at optimal point
        if exit_price is None:
            if direction == "morning_star":
                max_day = df.loc[window_dates]["High"].idxmax()
                exit_price = df.loc[max_day]["High"]
                exit_date = max_day
                reason = "max_window_exit"
            elif direction == "evening_star":
                min_day = df.loc[window_dates]["Low"].idxmin()
                exit_price = df.loc[min_day]["Low"]
                exit_date = min_day
                reason = "min_window_exit"

        if exit_price is None or exit_date is None:
            continue

        # Profit/loss
        profit_loss = (
            exit_price - entry_price if direction == "morning_star"
            else entry_price - exit_price
        )
        total_profit_loss += profit_loss

        trades.append({
            'entry_date': entry_date,
            'entry_price': entry_price,
            'exit_date': exit_date,
            'exit_price': exit_price,
            'target_price': target_price,
            'direction': direction,
            'profit_loss': profit_loss,
            'exit_reason': reason,
            'stoploss': row.get("stoploss", None)
        })

    return total_profit_loss, trades


In [64]:
star_df = detect_star(data)
total_pl, trades = calculate_profit_loss_star(data, star_df, target_pct=2.0, hold_days=10)

print(f"Total Profit/Loss from Morning & Evening Star: {total_pl:.2f}")
for trade in trades:
    print(
        f"{trade['direction']} | Entry: {trade['entry_date'].date()} at {trade['entry_price']:.2f}, "
        f"Exit: {trade['exit_date'].date()} at {trade['exit_price']:.2f}, "
        f"Target: {trade['target_price']:.2f}, Reason: {trade['exit_reason']}, "
        f"P/L: {trade['profit_loss']:.2f}"
    )


Total Profit/Loss from Morning & Evening Star: 0.00
